In [1]:
#!pip install fair-esm torch_geometric

In [2]:
import esm
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
from torch_geometric.utils import negative_sampling
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
import pickle, os

In [3]:
# --- ESM model ---
esm_model, esm_alphabet = esm.pretrained.esm2_t6_8M_UR50D()  # Lightweight for speed
batch_converter = esm_alphabet.get_batch_converter()
esm_model.eval()

def embed_sequence_esm(seq):
    seq = seq.replace(" ", "").upper()
    batch_labels, batch_strs, batch_tokens = batch_converter([("protein", seq)])
    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[6], return_contacts=False)
    token_reps = results["representations"][6][0, 1:len(seq)+1].mean(0)
    return token_reps.cpu().numpy()

In [4]:
# Load dataset
# -----------------------------
df = pd.read_csv("df_actions_27k.csv")

In [5]:
df.head()

,sequence_a,sequence_b,item_id_a,item_id_b,mode,is_directional,a_is_acting,score
0,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGER...,9606.ENSP00000000233,9606.ENSP00000250971,reaction,t,t,900
1,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGER...,9606.ENSP00000000233,9606.ENSP00000250971,reaction,t,f,900
2,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MTECFLPPTSSPSEHRRVEHGSGLTRTPSSEEISPTKFPGLYRTGE...,9606.ENSP00000000233,9606.ENSP00000019317,activation,f,f,175
3,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MQQAPQPYEFFSEENSPKWRGLLVSALRKVQEQVHPTLSANEESLY...,9606.ENSP00000000233,9606.ENSP00000216373,reaction,f,f,161
4,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MAMAEGERTECAEPPRDEPPADGALKRAEELKTQANDYFKAKDYEN...,9606.ENSP00000000233,9606.ENSP00000012443,catalysis,t,f,155


In [6]:
# Map mode to edge type (int)
df['edge_type'] = pd.factorize(df['mode'])[0]
mode_to_int = dict(zip(df['mode'], df['edge_type']))

In [7]:
# --- Unique proteins and sequences ---
proteins_a = df[['item_id_a', 'sequence_a']].rename(columns={'item_id_a': 'item_id', 'sequence_a': 'sequence'})
proteins_b = df[['item_id_b', 'sequence_b']].rename(columns={'item_id_b': 'item_id', 'sequence_b': 'sequence'})
all_proteins = pd.concat([proteins_a, proteins_b]).drop_duplicates("item_id").set_index("item_id")
protein_to_idx = {pid: i for i, pid in enumerate(all_proteins.index)}

In [8]:
# Check sample sequences before embedding
print("\n🔍 Sample sequences:")
print(all_proteins["sequence"].dropna().head(5))

# Redefine embed_sequence_esm with logging
def embed_sequence_esm(seq):
    try:
        seq = seq.replace(" ", "").upper()
        if not seq or not isinstance(seq, str) or len(seq) < 5:
            print("❌ Invalid sequence:", seq)
            return None

        batch_labels, batch_strs, batch_tokens = batch_converter([("protein", seq)])
        with torch.no_grad():
            results = esm_model(batch_tokens, repr_layers=[6], return_contacts=False)
        token_reps = results["representations"][6][0, 1:len(seq)+1]
        return token_reps.mean(0).cpu().numpy()
    except Exception as e:
        print(f"❌ Failed to embed sequence: {e}")
        return None

# Embedding loop with logging
protein_embeddings = {}
for pid in tqdm(all_proteins.index, desc="Embedding proteins"):
    try:
        seq = all_proteins.loc[pid, "sequence"]
        if not isinstance(seq, str) or len(seq.strip()) == 0:
            print(f"⚠️ Skipping {pid}: Empty or invalid sequence")
            continue
        embedding = embed_sequence_esm(seq)
        if embedding is not None:
            protein_embeddings[pid] = embedding
            print(f"✅ Embedded {pid}")
        else:
            print(f"⚠️ Skipped {pid}: Embedding returned None")
    except Exception as e:
        print(f"❌ Error embedding {pid}: {e}")

print(f"\n✅ Total embeddings generated: {len(protein_embeddings)}")



🔍 Sample sequences:
item_id
9606.ENSP00000000233    MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...
9606.ENSP00000000412    MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...
9606.ENSP00000000442    MSSQVVGIEPLYIKAEPASPDSPKGSSETETEPPVALAPGPAPTRC...
9606.ENSP00000001008    MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...
9606.ENSP00000001146    MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
Name: sequence, dtype: object


Embedding proteins:   0%|                                                             | 2/1690 [00:00<02:23, 11.76it/s]

✅ Embedded 9606.ENSP00000000233
✅ Embedded 9606.ENSP00000000412


Embedding proteins:   0%|▏                                                            | 4/1690 [00:00<03:53,  7.22it/s]

✅ Embedded 9606.ENSP00000000442
✅ Embedded 9606.ENSP00000001008


...

✅ Embedded 9606.ENSP00000251102


Embedding proteins: 100%|█████████████████████████████████████████████████████████▉| 1689/1690 [09:58<00:00,  1.88it/s]

✅ Embedded 9606.ENSP00000262210


Embedding proteins: 100%|██████████████████████████████████████████████████████████| 1690/1690 [09:58<00:00,  2.82it/s]

✅ Embedded 9606.ENSP00000259371

✅ Total embeddings generated: 1690


In [9]:
# cache_path = "esm2_embeddings_linkprediction_rgcn.pkl"

# if os.path.exists(cache_path):
#     with open(cache_path, "rb") as f:
#         protein_embeddings = pickle.load(f)
# else:
#     protein_embeddings = {}
#     for pid in tqdm(all_proteins.index, desc="Embedding proteins"):
#         try:
#             seq = all_proteins.loc[pid, "sequence"]
#             if not isinstance(seq, str) or len(seq.strip()) == 0:
#                 print(f"⚠️ Skipping {pid}: Empty or invalid sequence")
#                 continue
#             embedding = embed_sequence_esm(seq)
#             protein_embeddings[pid] = embedding
#         except Exception as e:
#             print(f"❌ Error embedding {pid}: {e}")
#     with open(cache_path, "wb") as f:
#         pickle.dump(protein_embeddings, f)

# print(f"✅ Total embeddings generated: {len(protein_embeddings)}")

In [10]:
# --- Node feature matrix ---
if len(protein_embeddings) == 0:
    raise ValueError("❌ No embeddings were generated. Check your embedding function or input sequences.")

embedding_dim = len(next(iter(protein_embeddings.values())))
x = np.zeros((len(protein_to_idx), embedding_dim), dtype=np.float32)
for pid, idx in protein_to_idx.items():
    x[idx] = protein_embeddings[pid]
x = torch.tensor(x, dtype=torch.float)

# --- Edges ---
edge_index = torch.tensor([
    [protein_to_idx[a] for a in df['item_id_a']],
    [protein_to_idx[b] for b in df['item_id_b']]
], dtype=torch.long)
edge_type = torch.tensor(df['edge_type'].values, dtype=torch.long)

# --- PyG data object ---
data = Data(x=x, edge_index=edge_index, edge_type=edge_type)

In [11]:
# --- Train/val/test split ---
ei = data.edge_index.numpy()
et = data.edge_type.numpy()
ei_train, ei_test, et_train, et_test = train_test_split(ei.T, et, test_size=0.2, random_state=42)
ei_train, ei_val, et_train, et_val = train_test_split(ei_train, et_train, test_size=0.1, random_state=42)

ei_train = torch.tensor(ei_train, dtype=torch.long).t()
ei_val = torch.tensor(ei_val, dtype=torch.long).t()
ei_test = torch.tensor(ei_test, dtype=torch.long).t()

et_train = torch.tensor(et_train, dtype=torch.long)
et_val = torch.tensor(et_val, dtype=torch.long)
et_test = torch.tensor(et_test, dtype=torch.long)

neg_train = negative_sampling(ei_train, num_nodes=data.num_nodes)
neg_val = negative_sampling(ei_val, num_nodes=data.num_nodes)
neg_test = negative_sampling(ei_test, num_nodes=data.num_nodes)

In [12]:
# --- RGCN model ---
class RGCNLinkPredictor(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_relations):
        super().__init__()
        self.conv1 = RGCNConv(in_dim, hidden_dim, num_relations)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations)

    def encode(self, x, edge_index, edge_type):
        x = F.relu(self.conv1(x, edge_index, edge_type))
        return self.conv2(x, edge_index, edge_type)

    def decode(self, z, edge_index):
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)

model = RGCNLinkPredictor(data.num_features, hidden_dim=64, out_dim=32, num_relations=len(mode_to_int))
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [13]:
# --- Training ---
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, ei_train, et_train)
    pos_score = model.decode(z, ei_train)
    neg_score = model.decode(z, neg_train)
    pos_labels = torch.ones_like(pos_score)
    neg_labels = torch.zeros_like(neg_score)
    loss = F.binary_cross_entropy_with_logits(pos_score, pos_labels) + \
           F.binary_cross_entropy_with_logits(neg_score, neg_labels)
    loss.backward()
    optimizer.step()
    return loss.item()

In [14]:
# --- Evaluation ---
def evaluate(ei_pos, et_pos, ei_neg):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, ei_pos, et_pos)
        pos_score = model.decode(z, ei_pos).sigmoid().cpu().numpy()
        neg_score = model.decode(z, ei_neg).sigmoid().cpu().numpy()
        y_true = np.concatenate([np.ones_like(pos_score), np.zeros_like(neg_score)])
        y_score = np.concatenate([pos_score, neg_score])
        roc = roc_auc_score(y_true, y_score)
        pr = average_precision_score(y_true, y_score)
        return roc, pr

In [15]:
# --- Train loop ---
for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        roc, pr = evaluate(ei_val, et_val, neg_val)
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")

# --- Final test evaluation ---
roc, pr = evaluate(ei_test, et_test, neg_test)
print(f"\n🧪 Test ROC-AUC: {roc:.4f} | Test PR-AUC: {pr:.4f}")

Epoch 010 | Loss: 1.2485 | Val ROC-AUC: 0.6687 | PR-AUC: 0.7107
Epoch 020 | Loss: 1.1479 | Val ROC-AUC: 0.7266 | PR-AUC: 0.7849
Epoch 030 | Loss: 1.1316 | Val ROC-AUC: 0.7382 | PR-AUC: 0.7984
Epoch 040 | Loss: 1.0990 | Val ROC-AUC: 0.7304 | PR-AUC: 0.7929
Epoch 050 | Loss: 1.0754 | Val ROC-AUC: 0.7109 | PR-AUC: 0.7798
Epoch 060 | Loss: 1.0548 | Val ROC-AUC: 0.6927 | PR-AUC: 0.7685
Epoch 070 | Loss: 1.0417 | Val ROC-AUC: 0.6808 | PR-AUC: 0.7599
Epoch 080 | Loss: 1.0308 | Val ROC-AUC: 0.6693 | PR-AUC: 0.7487
Epoch 090 | Loss: 1.0211 | Val ROC-AUC: 0.6571 | PR-AUC: 0.7362
Epoch 100 | Loss: 1.0132 | Val ROC-AUC: 0.6452 | PR-AUC: 0.7231
Epoch 110 | Loss: 1.0064 | Val ROC-AUC: 0.6363 | PR-AUC: 0.7116
Epoch 120 | Loss: 1.0006 | Val ROC-AUC: 0.6283 | PR-AUC: 0.7009
Epoch 130 | Loss: 0.9955 | Val ROC-AUC: 0.6234 | PR-AUC: 0.6941
Epoch 140 | Loss: 0.9909 | Val ROC-AUC: 0.6187 | PR-AUC: 0.6881
Epoch 150 | Loss: 0.9867 | Val ROC-AUC: 0.6201 | PR-AUC: 0.6891
Epoch 160 | Loss: 0.9827 | Val ROC-AUC: 

In [16]:
# Save model
torch.save(model.state_dict(), "rgcn_esm2_linkpredictor.pth")